In [5]:
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)
from client import Client
from fedavg import Fedavg
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils import shuffle
from supported_modles import Supported_modles
import random
import utils

In [25]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [26]:
selected_model = Supported_modles.logistic_regression

In [27]:
clients = utils.set_data()

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.


In [62]:
clients = utils.set_data()
f1_before = []
for client in clients:
    client.downsample_data(['sbytes','dbytes','sttl','dttl','spkts','dpkts'])
    client.split_data()
    client.prep_data()
    print(client.y.mean())
    client.train_model(selected_model)
    f1_before.append(client.f1)

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.
0.5520149253731343
0.5533582089552239
0.5540298507462686
0.5455223880597015
0.5515305440449138


In [29]:
f1_before

[0.7728169708929452,
 0.763525527051054,
 0.7711676129350779,
 0.767365496337459,
 0.7704644169729369]

In [56]:
selected_model = Supported_modles.SGD_classifier

In [57]:
fedavg = Fedavg("global",0.1)
fedavg.init_global_model(selected_model, clients[0].model,6)

[Errno 98] Address already in use
Waitiing for a Connection..


In [58]:
model = clients[0].model
clients = utils.set_data()
for client in clients:
    client.downsample_data(['sbytes','dbytes','sttl','dttl','spkts','dpkts'])
    client.split_data()
    client.init_empty_model(selected_model,mod)

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.


In [59]:
number_of_rounds = 40
batch_size = 0.2
epochs = 20
prep = StandardScaler() 

for _ in range(number_of_rounds):
    print(f'Starting new round!')

    applicable_clients = random.sample((clients), random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        # print(f'Client name: {client.name}')

        X = client.x
        y = client.y

        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=batch_size, stratify=y, random_state=random.randint(0,10))
       
        X_train = prep.fit_transform(X_train)

        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        fedavg.load_global_model(client.model, selected_model)
        fedavg.train_local_agent(X_train, y_train, client.model, epochs, sample_weights, selected_model)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    # round_weights = weights
    print(round_weights)
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

Starting new round!
[1.]
Starting new round!
[0.35823755 0.16044061 0.32088123 0.16044061]
Starting new round!
[0.33333333 0.33333333 0.16666667 0.16666667]
Starting new round!
[0.16044061 0.16044061 0.32088123 0.35823755]
Starting new round!
[0.38220194 0.42669709 0.19110097]
Starting new round!
[0.30870821 0.27651671 0.27651671 0.13825836]
Starting new round!
[0.23624824 0.52750353 0.23624824]
Starting new round!
[0.5 0.5]
Starting new round!
[1.]
Starting new round!
[0.2 0.4 0.4]
Starting new round!
[1.]
Starting new round!
[1.]
Starting new round!
[0.33333333 0.66666667]
Starting new round!
[0.4 0.2 0.4]
Starting new round!
[0.5 0.5]
Starting new round!
[0.47249647 0.52750353]
Starting new round!
[0.5 0.5]
Starting new round!
[0.4 0.4 0.2]
Starting new round!
[1.]
Starting new round!
[0.5  0.25 0.25]
Starting new round!
[0.13825836 0.27651671 0.27651671 0.30870821]
Starting new round!
[0.16044061 0.16044061 0.32088123 0.35823755]
Starting new round!
[0.30932595 0.69067405]
Starting

In [60]:
f1_fedavg = []
for client in clients:
    x = prep.transform(client.x_test)
    # x = client.x_test
    y_hat = fedavg.model.predict(x)
    f1_fedavg.append(f1_score(client.y_test,y_hat))
print(f1_fedavg)

[0.7859208045254557, 0.7884953529264004, 0.7826306488260539, 0.789553752535497, 0.783642626958551]


In [61]:
i = 0
for client in clients:
    print(f'Difference {f1_fedavg[i]-f1_before[i]}')
    i += 1

Difference 0.013103833632510442
Difference 0.024969825875346352
Difference 0.011463035890976081
Difference 0.022188256198037992
Difference 0.013178209985614142


In [198]:
selected_model = Supported_modles.gradient_boosting_classifier
client = utils.centralized_data()
client.downsample_data(['sbytes','dbytes','sttl','dttl','spkts','dpkts'])
client.split_data()
client.train_model(selected_model)
print(client.f1)

Creating node1 on 0.0.0.0:11111.
0.8913590604026845
